In [341]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Input,Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import GRU, Dense

In [342]:
df4=pd.read_csv(r"C:\Users\Jaya Teja\Stock_Sentiment_Analysis_Using_Machine_Learning_Open_Project\Finance_Project\df4.csv")

In [343]:
df4=df4.drop("Unnamed: 0",axis=1)

In [344]:
df4.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Label,Company,clean_title
0,2022-01-03,177.830002,182.880005,177.710007,182.009995,179.481140,104487900.0,1,1.0,strong apple 2022 product line could look like...
1,2022-01-03,382.583344,400.356659,378.679993,399.926666,399.926666,103931400.0,1,3.0,lg energy solution start taking order could so...
2,2022-01-03,167.550003,170.703506,166.160507,170.404495,170.404495,63520000.0,1,2.0,updating 2021 call suggesting strategic positi...
3,2022-01-04,182.630005,182.940002,179.119995,179.699997,177.203217,99310400.0,0,1.0,3 trillion beginning apple apple car come prep...
4,2022-01-04,396.516663,402.666656,374.350006,383.196655,383.196655,100248300.0,0,3.0,tesla open showroom xinjiang chinese region as...


In [345]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1746 non-null   object 
 1   Open         1746 non-null   float64
 2   High         1746 non-null   float64
 3   Low          1746 non-null   float64
 4   Close        1746 non-null   float64
 5   Adj Close    1746 non-null   float64
 6   Volume       1746 non-null   float64
 7   Label        1746 non-null   int64  
 8   Company      1746 non-null   float64
 9   clean_title  1746 non-null   object 
dtypes: float64(7), int64(1), object(2)
memory usage: 136.5+ KB


In [346]:
df4['Date'] = pd.to_datetime(df4['Date'])

In [347]:
df4['year'] = df4['Date'].dt.year
df4['month'] = df4['Date'].dt.month
df4['day'] = df4['Date'].dt.day
df4['dayofweek'] = df4['Date'].dt.dayofweek  # Monday=0, Sunday=6
df4['dayofyear'] = df4['Date'].dt.dayofyear
df4['is_month_start'] = df4['Date'].dt.is_month_start.astype(int)
df4['is_month_end'] = df4['Date'].dt.is_month_end.astype(int)

In [348]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1746 non-null   datetime64[ns]
 1   Open            1746 non-null   float64       
 2   High            1746 non-null   float64       
 3   Low             1746 non-null   float64       
 4   Close           1746 non-null   float64       
 5   Adj Close       1746 non-null   float64       
 6   Volume          1746 non-null   float64       
 7   Label           1746 non-null   int64         
 8   Company         1746 non-null   float64       
 9   clean_title     1746 non-null   object        
 10  year            1746 non-null   int64         
 11  month           1746 non-null   int64         
 12  day             1746 non-null   int64         
 13  dayofweek       1746 non-null   int64         
 14  dayofyear       1746 non-null   int64         
 15  is_m

## Modle on Sentimen Intensity Analyser

In [349]:
sentiment_scores=pd.read_pickle("sentiment_scores.pkl")

In [350]:
neg_sentiment=sentiment_scores.apply(lambda x: x['neg'])
neg_sentiment.name='neg'
neu_sentiment=sentiment_scores.apply(lambda x: x['neu'])
neu_sentiment.name='neu'
pos_sentiment=sentiment_scores.apply(lambda x:x['pos'])
pos_sentiment.name='pos'
comp_sentiment=sentiment_scores.apply(lambda x: x['compound'])
comp_sentiment.name='comp'

In [351]:
neg_sentiment

0       0.022
1       0.055
2       0.048
3       0.018
4       0.056
        ...  
1741    0.050
1742    0.034
1743    0.063
1744    0.077
1745    0.097
Name: neg, Length: 1746, dtype: float64

In [352]:
req_col=["Open","High","Low","Close","Volume","year","month","day","dayofweek","dayofyear","is_month_start","is_month_end"]
final_df=pd.concat([df4[req_col],neg_sentiment,neu_sentiment,pos_sentiment,comp_sentiment],axis=1)


In [353]:
final_df

,Open,High,Low,Close,Volume,year,month,day,dayofweek,dayofyear,is_month_start,is_month_end,neg,neu,pos,comp
0,177.830002,182.880005,177.710007,182.009995,104487900.0,2022,1,3,0,3,0,0,0.022,0.781,0.197,0.9941
1,382.583344,400.356659,378.679993,399.926666,103931400.0,2022,1,3,0,3,0,0,0.055,0.813,0.132,0.9900
2,167.550003,170.703506,166.160507,170.404495,63520000.0,2022,1,3,0,3,0,0,0.048,0.698,0.254,0.9904
3,182.630005,182.940002,179.119995,179.699997,99310400.0,2022,1,4,1,4,0,0,0.018,0.818,0.165,0.9849
4,396.516663,402.666656,374.350006,383.196655,100248300.0,2022,1,4,1,4,0,0,0.056,0.784,0.160,0.9821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,169.529999,170.610001,168.149994,169.889999,50558300.0,2024,4,25,3,116,0,0,0.050,0.747,0.203,0.9468
1742,169.679993,173.919998,166.320007,173.669998,49249400.0,2024,4,25,3,116,0,0,0.034,0.684,0.282,0.9897
1743,177.800003,180.820007,176.130005,179.619995,43919800.0,2024,4,26,4,117,0,0,0.063,0.720,0.217,0.9762
1744,169.880005,171.339996,169.179993,169.300003,44838400.0,2024,4,26,4,117,0,0,0.077,0.793,0.130,0.5733


In [354]:
test_size=51

In [355]:
final_y=final_df["Close"]

In [356]:
final_y

0       182.009995
1       399.926666
2       170.404495
3       179.699997
4       383.196655
           ...    
1741    169.889999
1742    173.669998
1743    179.619995
1744    169.300003
1745    168.289993
Name: Close, Length: 1746, dtype: float64

In [357]:
final_df=final_df.drop("Close",axis=1)

In [358]:
X_test=final_df[-test_size:]
y_test=final_y[-test_size:]

In [359]:
comb_train=pd.concat([final_df.iloc[:-test_size,:],final_y[:-test_size]],axis=1)
comb_train

,Open,High,Low,Volume,year,month,day,dayofweek,dayofyear,is_month_start,is_month_end,neg,neu,pos,comp,Close
0,177.830002,182.880005,177.710007,104487900.0,2022,1,3,0,3,0,0,0.022,0.781,0.197,0.9941,182.009995
1,382.583344,400.356659,378.679993,103931400.0,2022,1,3,0,3,0,0,0.055,0.813,0.132,0.9900,399.926666
2,167.550003,170.703506,166.160507,63520000.0,2022,1,3,0,3,0,0,0.048,0.698,0.254,0.9904,170.404495
3,182.630005,182.940002,179.119995,99310400.0,2022,1,4,1,4,0,0,0.018,0.818,0.165,0.9849,179.699997
4,396.516663,402.666656,374.350006,100248300.0,2022,1,4,1,4,0,0,0.056,0.784,0.160,0.9821,383.196655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,169.080002,169.339996,168.229996,49329500.0,2024,4,2,1,93,0,0,0.094,0.765,0.142,0.6908,168.839996
1691,179.070007,180.789993,178.380005,32611500.0,2024,4,2,1,93,0,0,0.053,0.719,0.228,0.9819,180.690002
1692,164.020004,168.820007,163.279999,82950100.0,2024,4,3,2,94,0,0,0.197,0.717,0.086,-0.9538,168.380005
1693,179.899994,182.869995,179.800003,31046600.0,2024,4,3,2,94,0,0,0.050,0.556,0.394,0.9974,182.410004


In [360]:
shuffled_df = comb_train.sample(frac=1, random_state=42)
shuffled_df

,Open,High,Low,Volume,year,month,day,dayofweek,dayofyear,is_month_start,is_month_end,neg,neu,pos,comp,Close
931,193.130005,195.289993,189.440002,123660000.0,2023,3,29,2,88,0,0,0.133,0.682,0.185,0.8020,193.880005
1398,142.020004,142.649994,139.839996,36235400.0,2023,11,9,3,313,0,0,0.075,0.680,0.244,0.9488,140.600006
1561,160.699997,161.729996,158.490005,45207400.0,2024,1,30,1,30,0,0,0.126,0.648,0.226,0.9827,159.000000
1006,104.040001,105.389999,103.309998,45345500.0,2023,5,4,3,124,0,0,0.055,0.661,0.285,0.9910,104.000000
506,129.500000,131.380005,126.389999,57429800.0,2022,9,2,4,245,0,0,0.097,0.694,0.209,0.8360,127.510002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,130.240005,131.399994,129.639999,35895400.0,2023,7,5,2,186,0,0,0.073,0.729,0.198,0.9698,130.380005
1294,174.550003,176.300003,173.860001,63047900.0,2023,9,21,3,264,0,0,0.032,0.941,0.027,-0.1280,173.929993
860,150.089996,150.339996,147.240005,48394200.0,2023,2,23,3,54,0,0,0.071,0.744,0.185,0.9607,149.399994
1459,194.199997,195.990005,193.669998,53377300.0,2023,12,8,4,342,0,0,0.110,0.741,0.149,0.8272,195.710007


In [361]:
X_train=shuffled_df.drop("Close",axis=1)
y_train=shuffled_df["Close"]

In [362]:
X_train

,Open,High,Low,Volume,year,month,day,dayofweek,dayofyear,is_month_start,is_month_end,neg,neu,pos,comp
931,193.130005,195.289993,189.440002,123660000.0,2023,3,29,2,88,0,0,0.133,0.682,0.185,0.8020
1398,142.020004,142.649994,139.839996,36235400.0,2023,11,9,3,313,0,0,0.075,0.680,0.244,0.9488
1561,160.699997,161.729996,158.490005,45207400.0,2024,1,30,1,30,0,0,0.126,0.648,0.226,0.9827
1006,104.040001,105.389999,103.309998,45345500.0,2023,5,4,3,124,0,0,0.055,0.661,0.285,0.9910
506,129.500000,131.380005,126.389999,57429800.0,2022,9,2,4,245,0,0,0.097,0.694,0.209,0.8360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,130.240005,131.399994,129.639999,35895400.0,2023,7,5,2,186,0,0,0.073,0.729,0.198,0.9698
1294,174.550003,176.300003,173.860001,63047900.0,2023,9,21,3,264,0,0,0.032,0.941,0.027,-0.1280
860,150.089996,150.339996,147.240005,48394200.0,2023,2,23,3,54,0,0,0.071,0.744,0.185,0.9607
1459,194.199997,195.990005,193.669998,53377300.0,2023,12,8,4,342,0,0,0.110,0.741,0.149,0.8272


In [363]:
# SVM Linear

In [249]:
# 4:55 start

In [364]:

scaler = StandardScaler()

In [365]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [366]:
svm_reg = SVR(kernel='linear', C=1, gamma='scale')
svm_reg.fit(X_train, y_train)

SVR(C=1, kernel='linear')

In [367]:
y_pred = svm_reg.predict(X_test)

In [368]:
mean_squared_error(y_test, y_pred)

2.807917724528105

In [369]:
r2_score(y_test, y_pred)

0.9745865731101075

In [370]:
yyy=svm_reg.predict(X_train)

In [371]:
mean_squared_error(y_train, yyy)

7.1044583410629425

In [372]:
r2_score(y_train, yyy)

0.9978144427945507

In [373]:
# SVM RBF

In [374]:
# svm_reg = SVR(kernel='rbf', C=1, gamma='scale')
# svm_reg.fit(X_train, y_train)

SVR(C=1)

In [375]:
# y_pred = svm_reg.predict(X_test)

In [376]:
# mean_squared_error(y_test, y_pred)

16.430676290352007

In [377]:
# r2_score(y_test, y_pred)

0.8512920136481122

In [378]:
# yyy=svm_reg.predict(X_train)

In [379]:
# mean_squared_error(y_train, yyy)

580.0204003339246

In [380]:
# r2_score(y_train, yyy)

0.8215672885390248

In [381]:
# LSTM

In [382]:
# model = Sequential()
# model.add(Input(shape=(15,)))  # Input shape: (15,)
# model.add(Reshape((1, 15)))  # Reshape to (batch_size, timesteps, input_dim)
# model.add(LSTM(100, return_sequences=True))
# model.add(LSTM(50))
# model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification


# optimizer = Adam(learning_rate=0.001)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [383]:
# model.fit(X_train, y_train, epochs=30, batch_size=8, validation_split=0.2)

Epoch 1/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.0000e+00 - loss: -200.6548 - val_accuracy: 0.0000e+00 - val_loss: -1999.2338
Epoch 2/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -2815.8826 - val_accuracy: 0.0000e+00 - val_loss: -4267.7163
Epoch 3/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0000e+00 - loss: -4706.8037 - val_accuracy: 0.0000e+00 - val_loss: -5701.4341
Epoch 4/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.0000e+00 - loss: -6130.7300 - val_accuracy: 0.0000e+00 - val_loss: -7011.1445
Epoch 5/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -7352.1357 - val_accuracy: 0.0000e+00 - val_loss: -8263.9902
Epoch 6/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -8732.5078 - val_accuracy: 0.0000e+00 - val_loss: -9489.4932
Epoch 7/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0000e+00 - loss: -9812.0479 - val_accuracy: 0.0000e+00 - val_loss: -1

In [384]:
# y_pred=model.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step


In [385]:
# mean_squared_error(y_test, y_pred)

29046.233707321975

In [386]:
# r2_score(y_test, y_pred)

-261.88673998516475

In [387]:
# GRU

In [389]:
# optimizer = Adam(learning_rate=0.001)
# batch_size = 8
# epochs = 30

In [390]:
# X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

# # Reshape test set: (53, 622) -> (53, 1, 622)
# X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [391]:
# model_gru_2hidden = Sequential()
# model_gru_2hidden.add(Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
# model_gru_2hidden.add(GRU(units=100, return_sequences=True))
# model_gru_2hidden.add(GRU(units=50))
# model_gru_2hidden.add(Dense(units=1, activation='sigmoid'))  # Assuming binary classification

# # Compile the model
# optimizer = Adam(learning_rate=0.001)
# model_gru_2hidden.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [392]:
# model_gru_2hidden.fit(X_train_reshaped, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)
                      

Epoch 1/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.0000e+00 - loss: -478.7467 - val_accuracy: 0.0000e+00 - val_loss: -3458.3203
Epoch 2/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -4061.6169 - val_accuracy: 0.0000e+00 - val_loss: -5549.4458
Epoch 3/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -6054.6201 - val_accuracy: 0.0000e+00 - val_loss: -7273.6069
Epoch 4/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -7636.3477 - val_accuracy: 0.0000e+00 - val_loss: -8912.2002
Epoch 5/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -9361.2256 - val_accuracy: 0.0000e+00 - val_loss: -10504.4287
Epoch 6/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -10992.7178 - val_accuracy: 0.0000e+00 - val_loss: -12072.0166
Epoch 7/30
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0000e+00 - loss: -12611.4551 - val_accuracy: 0.0000e+00 - val_loss

In [393]:
# y_pred=model_gru_2hidden.predict(X_test_reshaped)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step


In [394]:
# r2_score(y_test, y_pred)

-261.88673998516475

In [395]:
# mean_squared_error(y_test, y_pred)

29046.233707321975

In [396]:
# yyy=model_gru_2hidden.predict(X_train_reshaped)

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [397]:
# mean_squared_error(y_train,yyy)

34048.258881088026

## Model On Pretrained Word2Vec

In [398]:
pre_trained_word2vec=pd.read_pickle("pre_trained_word2vec.pkl")

In [399]:
pre_trained_word2vec=pd.DataFrame(pre_trained_word2vec)

In [402]:
final_df=pd.concat([df4[req_col],features_df],axis=1)

In [403]:
final_df.head()

,Open,High,Low,Close,Volume,year,month,day,dayofweek,dayofyear,...,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,feature300
0,177.830002,182.880005,177.710007,182.009995,104487900.0,2022,1,3,0,3,...,-0.037207,0.050193,-0.079310,-0.014028,0.012350,-0.026454,-0.005330,-0.006079,0.000593,0.028662
1,382.583344,400.356659,378.679993,399.926666,103931400.0,2022,1,3,0,3,...,-0.027100,0.065207,-0.118229,0.026111,0.006336,-0.039052,-0.044905,0.007134,-0.000578,0.013845
2,167.550003,170.703506,166.160507,170.404495,63520000.0,2022,1,3,0,3,...,-0.036802,0.060348,-0.097775,-0.006845,0.014355,-0.033606,0.029684,0.015215,0.007803,0.019330
3,182.630005,182.940002,179.119995,179.699997,99310400.0,2022,1,4,1,4,...,-0.043618,0.039698,-0.070958,-0.000070,0.010268,-0.032488,-0.024274,-0.000923,0.018238,0.011740
4,396.516663,402.666656,374.350006,383.196655,100248300.0,2022,1,4,1,4,...,-0.025634,0.069249,-0.136903,0.030683,0.020873,-0.009336,-0.029908,0.004478,0.026997,0.008298


In [404]:
final_y=final_df["Close"]
final_y

0       182.009995
1       399.926666
2       170.404495
3       179.699997
4       383.196655
           ...    
1741    169.889999
1742    173.669998
1743    179.619995
1744    169.300003
1745    168.289993
Name: Close, Length: 1746, dtype: float64

In [405]:
final_df=final_df.drop("Close",axis=1)

In [406]:
X_test=final_df[-test_size:]
y_test=final_y[-test_size:]

In [407]:
comb_train=pd.concat([final_df.iloc[:-test_size,:],final_y[:-test_size]],axis=1)
comb_train

,Open,High,Low,Volume,year,month,day,dayofweek,dayofyear,is_month_start,...,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,feature300,Close
0,177.830002,182.880005,177.710007,104487900.0,2022,1,3,0,3,0,...,0.050193,-0.079310,-0.014028,0.012350,-0.026454,-0.005330,-0.006079,0.000593,0.028662,182.009995
1,382.583344,400.356659,378.679993,103931400.0,2022,1,3,0,3,0,...,0.065207,-0.118229,0.026111,0.006336,-0.039052,-0.044905,0.007134,-0.000578,0.013845,399.926666
2,167.550003,170.703506,166.160507,63520000.0,2022,1,3,0,3,0,...,0.060348,-0.097775,-0.006845,0.014355,-0.033606,0.029684,0.015215,0.007803,0.019330,170.404495
3,182.630005,182.940002,179.119995,99310400.0,2022,1,4,1,4,0,...,0.039698,-0.070958,-0.000070,0.010268,-0.032488,-0.024274,-0.000923,0.018238,0.011740,179.699997
4,396.516663,402.666656,374.350006,100248300.0,2022,1,4,1,4,0,...,0.069249,-0.136903,0.030683,0.020873,-0.009336,-0.029908,0.004478,0.026997,0.008298,383.196655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,169.080002,169.339996,168.229996,49329500.0,2024,4,2,1,93,0,...,0.071556,-0.102970,-0.002672,-0.003684,-0.041685,-0.065024,-0.022977,0.012978,0.033470,168.839996
1691,179.070007,180.789993,178.380005,32611500.0,2024,4,2,1,93,0,...,0.085056,-0.115396,0.025746,0.021550,-0.054409,-0.017465,-0.023396,-0.017140,0.005757,180.690002
1692,164.020004,168.820007,163.279999,82950100.0,2024,4,3,2,94,0,...,0.091984,-0.139240,0.031161,0.014353,-0.034526,-0.048514,-0.001056,0.021466,0.026095,168.380005
1693,179.899994,182.869995,179.800003,31046600.0,2024,4,3,2,94,0,...,0.069081,-0.122761,0.041219,0.015562,-0.029679,-0.021318,-0.008493,0.001870,0.018600,182.410004


In [408]:
shuffled_df = comb_train.sample(frac=1, random_state=42)
shuffled_df

,Open,High,Low,Volume,year,month,day,dayofweek,dayofyear,is_month_start,...,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,feature300,Close
931,193.130005,195.289993,189.440002,123660000.0,2023,3,29,2,88,0,...,0.056397,-0.133346,0.016269,0.006206,-0.021397,-0.046383,0.012801,0.027904,0.029257,193.880005
1398,142.020004,142.649994,139.839996,36235400.0,2023,11,9,3,313,0,...,0.062648,-0.091701,0.012577,0.024324,-0.035703,-0.011585,0.003525,0.025229,-0.001169,140.600006
1561,160.699997,161.729996,158.490005,45207400.0,2024,1,30,1,30,0,...,0.036798,-0.097982,-0.006071,0.007805,-0.046946,-0.000371,-0.025581,0.004604,-0.012706,159.000000
1006,104.040001,105.389999,103.309998,45345500.0,2023,5,4,3,124,0,...,0.077400,-0.112755,0.021999,0.055317,-0.028893,0.019799,-0.011909,0.004803,0.006767,104.000000
506,129.500000,131.380005,126.389999,57429800.0,2022,9,2,4,245,0,...,0.054824,-0.147058,0.025691,0.018865,-0.021517,0.000567,-0.011283,0.012656,0.009158,127.510002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,130.240005,131.399994,129.639999,35895400.0,2023,7,5,2,186,0,...,0.080669,-0.117940,0.051831,0.026700,-0.033837,-0.036460,-0.002618,-0.004655,0.008348,130.380005
1294,174.550003,176.300003,173.860001,63047900.0,2023,9,21,3,264,0,...,0.058117,-0.077643,-0.020928,0.044710,-0.037677,-0.004288,0.007928,0.020461,0.015157,173.929993
860,150.089996,150.339996,147.240005,48394200.0,2023,2,23,3,54,0,...,0.066301,-0.077316,0.007268,0.006318,-0.056321,-0.026234,-0.007728,0.007498,0.018985,149.399994
1459,194.199997,195.990005,193.669998,53377300.0,2023,12,8,4,342,0,...,0.044330,-0.088014,0.005259,-0.009609,-0.029580,-0.035272,0.001940,0.002409,-0.000055,195.710007


In [409]:
X_train=shuffled_df.drop("Close",axis=1)
y_train=shuffled_df["Close"]

In [218]:
X_train.columns=X_train.columns.astype(str)

In [411]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [413]:
svm_reg = SVR(kernel='linear', C=1, gamma='scale')
svm_reg.fit(X_train, y_train)

SVR(C=1, kernel='linear')

In [414]:
y_pred = svm_reg.predict(X_test)

In [415]:
mean_squared_error(y_test, y_pred)

4.1397587687322

In [416]:
r2_score(y_test, y_pred)

0.9625325714169038

In [417]:
yyy=svm_reg.predict(X_train)

In [418]:
mean_squared_error(y_train, yyy)

6.621220500237811